**Teymisverkefni 2**

Í teymisverkefni-2 er markmiðið að læra leikáætlun með því að spila á móti annarri leikáætlun sem er líka að læra (e. both sides are learning). Sjá umræðu í Sutton and Barto (fyrsta kafla) um self-play og tic-tac-toe. 

Hvert teymi skilar **teymiX.npy** skrá með stefnu $\pi(s,a)$ þar sem $s$ er skilgreint  með Zobrist hashing (sjá neðar) ásamt læsilegri og auðskiljanlegri útfærslu á reikniriti.

Frjálst er að velja "reinforcement learning" reiknirit sem við höfum fjallað um í þessari námslotu. Sem dæmi, það má vera TD($0$), MC, TD($\lambda$), $Q-$learning, SARSA($\lambda$), expected SARSA($\lambda$). Svo þarf að spá í hvernig *exploration* er útfært.

Áður en þið byrjið skulum við ræða eftirfarandi æfingar úr Sutton og Barto:

  1. (Exercise 1.1): **Self-Play** Suppose, instead of playing against a random opponent, the reinforcement learning algorithm described above played against itself, with both sides learning. What do you think would happen in this case? Would it learn a different policy for selecting moves?

  2. (Exercise 1.2): **Symmetries** Some Connect-4 positions appear different but are really the same because of symmetries. How might we amend the learning process described above to take advantage of this? In what ways would this change improve the learning process? Now think again. Suppose the opponent did not take advantage of symmetries. In that case, should we? Is it true, then, that symmetrically equivalent positions should necessarily have the same value?

  3. Exercise 1.3: **Greedy Play** Suppose the reinforcement learning player was greedy, that is, it always played the move that brought it to the position that it rated the best. Might it learn to play better, or worse, than a nongreedy player? What problems might occur?

  4. Exercise 1.4: **Learning from Exploration** Suppose learning updates occurred after all moves, including exploratory moves. If the step-size parameter is appropriately reduced over time (but not the tendency to explore), then the state values would converge to a different set of probabilities. What (conceptually) are the two sets of probabilities computed when we do, and when we do not, learn from exploratory moves? Assuming that we do continue to make exploratory moves, which set of probabilities might be better to learn? Which would result in more wins?

  5. *Exercise 1.5*: **Other Improvements** Can you think of other ways to improve the reinforce- ment learning player? Can you think of any better way to solve the Connect-3 problem as posed?

In [ ]:
import numpy as np

**Connect-3 a mini version of Connect-4 on a 5x5 board**

The following code implements the game Connect-3 on a 5-by-5 board. We will walk through this code in class. I am open for any improvement you may suggest for making the code faster. The Zobrist hashing table is randomly generated using the seed 42. Don't change how we generate the hashed states, I aim to let your policy compete against policies generated by other teams. This final exercise, will be used as input to our discussion on Sutton and Barto's exercise 1.1.

In [ ]:
# two player game (1) versus (2)
getotherplayer = lambda p : 3-p # returns the other player
# the initial empty board, in matrix board we store legal indices in board[:,-1]
def iState(n = 5, m = 5):
  return np.zeros((n,m+1), dtype=np.uint16)
# perform move for player on board
def Action(board, move, player):
  if board[move,-2] > 0:
    print("illegal move ", move, " for board ", np.flipud(board.T))
    raise
  else:
    board[move,board[move,-1]] = player # place the disc on board
    board[move,-1] += 1 # next legal drop
  return board
# determine if terminal board state, assuming last move was made by player p
def terminal(board, p, i, j, n = 5, m = 5):
  # Horizontal
  t = -min(2, i)
  count = 0
  while (t <= min(2, n-1-i) and count < 3):
    if board[i+t,j] == p:
      count += 1
    else:
      count = 0
    t += 1
  if count == 3:
    return True
  
  # Vertical
  t = -min(2, j)
  count = 0
  while (t <= min(2, m-1-j) and count < 3):
    if board[i,j+t] == p:
      count += 1
    else:
      count = 0
    t += 1
  if count == 3:
    return True
  
  # Main diagonal
  t = -min(2, min(i, j))
  count = 0
  while (t <= min(2, min(n-1-i, m-1-j)) and count < 3):
    if board[i+t,j+t] == p:
      count += 1
    else:
      count = 0
    t += 1
  if count == 3:
    return True

  # Antidiagonal
  t = -min(2, min(i, m-1-j))
  count = 0
  while (t <= min(2, min(n-1-i, j)) and count < 3):
    if board[i+t,j-t] == p:
      count += 1
    else:
      count = 0
    t += 1
  if count == 3:
    return True
  return False

# Some pretty way of displaying the board in the terminal
def pretty_print(board, n = 5, m = 5, symbols = " XO"):
  for num in range(1, n+1):
    print(" " + str(num) + " ", end = " ")
  print()
  for j in range(m):
    for i in range(n):
      print(" " + symbols[board[i,m-1-j]] + " ", end = " ")
    print("")
# let's simulate a single game using pure random play, i.e. demonstrate an episode!
def connect3():
  S = iState() # initial board state
  p = 1 # first player to move (other player is 2)
  a = np.random.choice(np.where(S[:,-2] == 0)[0], 1) # first move is random
  S = Action(S, int(a), p) # force first move to be random
  p = getotherplayer(p) # other player's turn
  while True:
    # XX
    # Look ahead til að athuga value á næstu 5 mögulegu stöðum
    # hash-a stöðu, xor-a við allt sem er í boði og skoða gildi þeirrar stöðu
    # taka það action sem fer í besta gildið, nema random með epsilon líkum

    a = np.random.choice(np.where(S[:,-2]==0)[0],1) # pure random policy
    if 0 == len(a): # check if a legal move was possible, else bail out
      return 0, S # its a draw, return 0 and board

    S = Action(S,int(a),p) # take action a and update the board state
    if terminal(S, p, int(a), int(S[a,-1] - 1)):
      return p, S # return the winning player and board
    p = getotherplayer(p) # other player's turn
  return 0, S # default is a draw

# run demo for random play policy:
winner, board = connect3()
symbols = " XO"
print(" winner is '", symbols[winner],"' final board is:\n")
pretty_print(board)

 winner is ' X ' final board is:

 1   2   3   4   5  
         O       O  
         O       O  
 X       X       X  
 X       X   X   O  
 O       O   X   X  


**Constructing value and action value tables using [Zobrist hashing](https://en.wikipedia.org/wiki/Zobrist_hashing):**

Please do not modify how the zobTable is generated.


In [ ]:
print(np.where(board[:,-2] == 0)[0])

[0 1 3]


In [ ]:
"""
# let's all use the same zobTable, so we set the random seed
np.random.seed(42)
zobTable = np.random.randint(1,2**(5*5)-1, size=(5,5,3), dtype = np.uint32)
# compute index from current board state
def computeHash(board, n = 5, m = 5):
  h = 0
  for i in range(n):
    for j in range(board[i,-1]):
      h ^= zobTable[i,j,board[i,j]]
  return h
"""
np.random.seed(42)
zobTable = np.random.randint(1,2**64-1, size=(5,5,3), dtype = np.uint64)
# compute index from current board state
def computeHash(board, n = 5, m = 5):
  h = np.uint64(0)
  for i in range(n):
    for j in range(board[i,-1]):
      h ^= zobTable[i,j,board[i,j]]
  return h

In [ ]:
maxHashValue = 2**(5*5)-2
#V = np.zeros(maxHashValue, dtype=np.int16)
V = {}

In [ ]:
def nextHash(old_hash, i, j, p):
  return old_hash ^ zobTable[i,j,p]

def getVal(h):
  if h in V:
    return V[h]
  return 0

# Save your policy, PI to file, see folder icon on left hand side to download!
#np.save("teymiX", PI)
#!ls

upd = [0, 0, 0]
def learn(greedy1 = False, greedy2 = False, pr = False):
  S = iState() # initial board state
  p = 1 # first player to move (other player is 2)
  a = np.random.choice(np.where(S[:,-2] == 0)[0], 1) # first move is random
  if pr:
    print(int(a), end="")
  S = Action(S, int(a), p) # force first move to be random
  p = getotherplayer(p) # other player's turn
  h = computeHash(S)
  b = pr
  ct = 1
  while True:
    ct +=  1
    a = np.where(S[:,-2] == 0)[0]
    if 0 == len(a): # check if a legal move was possible, else bail out
      V[h] = 1
      if pr:
        print("draw, moves = {}".format(ct))
      return 0 # its a draw, return 0 and board
    vals = [getVal(nextHash(h, i, S[i,-1], p)) for i in a]
    #vals = [V[nextHash(h, i, S[i,-1], p)] for i in a]
    v = getVal(h)
    if v == 0:
      if -1 in vals:
        upd[0] += 1
        V[h] = 2
      elif 0 not in vals and 1 not in vals:
        upd[1] += 1
        V[h] = -1
      elif 0 not in vals:
        upd[2] += 1
        V[h] = 1
    if (p == 1 and not greedy1) or (p == 2 and not greedy2):
      # random policy
      a = np.random.choice(a, 1)[0]
    else:
      # greedy policy
      a = a[np.argmin(vals)]

    S = Action(S, a, p) # take action a and update the board state
    h1 = nextHash(h, a, S[a,-1] - 1, p)
    if terminal(S, p, a, S[a,-1] - 1):
      if (v == 0):
        V[h] = 2
        V[h1] = -1
      if pr:
        print("w = {}, moves = {}".format(p, ct))
        #pretty_print(S)
      return p
    if pr:
      if b:
        print(", {}: ".format(a), end="")
        b = False
      print(getVal(h), end=" ")
    h = h1 # Update the hash value
    p = getotherplayer(p) # other player's turn

for i in range(10):
  upd = [0, 0, 0]
  for j in range(10000):
    learn()
  print("{}: {}, {}, {}".format(i, upd[0], upd[1], upd[2]))
  for j in range(5):
    learn(greedy1=True, greedy2=True, pr=True)
w = 0
l = 0
for j in range(10000):
  c = learn(greedy1=True)
  if c == 1:
    w += 1
  elif c == 2:
    l += 1
print("Player 1 greedy:\nWins: {}\nLosses: {}\n".format(w, l))
w = 0
l = 0
for j in range(10000):
  c = learn(greedy2=True)
  if c == 1:
    w += 1
  elif c == 2:
    l += 1
print("Player 2 greedy:\nWins: {}\nLosses: {}\n".format(l, w))
w = 0
l = 0
for j in range(10000):
  c = learn(greedy1=True, greedy2=True)
  if c == 1:
    w += 1
  elif c == 2:
    l += 1
print("Both greedy:\nPlayer 1 win: {}\nPlayer 2 win: {}".format(w, l))

0: 412, 86, 0
2, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
4, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
2, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
4, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
1: 398, 82, 1
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
1, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
0, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
2, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
2: 391, 82, 0
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
0, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
1, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
4, 2: 2 -1 2 -1 2 -1 2 -1 w = 2, moves = 10
1, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
3: 380, 91, 1
4, 2: 2 -1 2 -1 2 -1 2 -1 w = 2, moves = 10
0, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
4, 2: 2 -1 2 -1 2 -1 2 -1 w = 2, moves = 10
0, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
4: 413, 93, 0
0, 2: 2 -1 2 -1 2 -1 w = 2, moves = 8
3, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9
1, 0: -1 2 -1 2 -1 2 -1 w = 1, moves = 9